In [28]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from cdlib import algorithms, evaluation, viz

In [30]:
# dataset loading for community only
edges = pd.read_csv('grouped_df_file2.csv')
edges[edges['Target'] == 'CUST28295610']

,Source,Target,counts,Amount,Risk Index,Weight
2102,EXTERNAL673277,CUST28295610,1,670.0,0.3,201.00
2427,CUST55380756,CUST28295610,1,396.0,0.3,118.80
2451,CUST62829842,CUST28295610,1,378.5,0.3,113.55
2454,EXTERNAL633225,CUST28295610,1,374.0,0.3,112.20
3213,CUST38721914,CUST28295610,1,92.0,0.3,27.60
3518,CUST56054611,CUST28295610,1,30.0,0.3,9.00
3676,CUST95706352,CUST28295610,1,8.0,0.3,2.40


In [31]:
edges[edges['Source'] == 'CUST28295610']

,Source,Target,counts,Amount,Risk Index,Weight
867,CUST28295610,CUST45674350,2,2664.0,1.0,2664.00
1990,CUST28295610,EXTERNAL315787,1,812.0,0.3,243.60
2156,CUST28295610,CUST88755324,1,614.5,0.3,184.35
2321,CUST28295610,EXTERNAL251130,1,477.0,0.3,143.10
2531,CUST28295610,CUST38566003,1,329.5,0.3,98.85
2673,CUST28295610,CUST21302276,1,261.0,0.3,78.30


In [32]:
edgelist_plot= nx.from_pandas_edgelist(edges, source='Source', target='Target', edge_attr=["Weight"])
np.random.seed(42)
communities = nx.community.louvain_communities(edgelist_plot, weight='Weight', seed=111)

In [33]:
print("No. of Communities: ",len(communities))

No. of Communities:  130


In [34]:
def compute_density(graph, community):
    community_subgraph = graph.subgraph(community)
    num_edges_within_community = community_subgraph.number_of_edges()
    num_nodes_within_community = community_subgraph.number_of_nodes()
    if num_nodes_within_community > 1:
        density = 2 * num_edges_within_community / (num_nodes_within_community * (num_nodes_within_community - 1))
        return density
    else:
        return 0

def compute_expansion(graph, community):
    external_nodes = set(community)
    internal_nodes = set(graph.nodes()) - external_nodes
    num_external_edges = sum(graph.degree(node) for node in external_nodes)
    num_internal_edges = sum(graph.degree(node) for node in internal_nodes)
    if num_internal_edges > 0:
        expansion = num_external_edges / num_internal_edges
        return expansion
    else:
        return 0

def compute_conductance(graph, community):
    external_nodes = set(community)
    internal_nodes = set(graph.nodes()) - external_nodes
    num_cut_edges = sum(1 for u, v in graph.edges() if (u in internal_nodes and v in external_nodes) or (u in external_nodes and v in internal_nodes))
    num_internal_edges = sum(graph.degree(node) for node in internal_nodes)
    if num_internal_edges > 0:
        conductance = num_cut_edges / (2 * num_internal_edges + num_cut_edges)
        return conductance
    else:
        return 0

# Assuming 'edgelist_plot' is your graph and 'communities' is a list of communities
lou_density_values = []
lou_expansion_values = []
lou_conductance_values = []

for community in communities:
    density = compute_density(edgelist_plot, community)
    expansion = compute_expansion(edgelist_plot, community)
    conductance = compute_conductance(edgelist_plot, community)
    lou_density_values.append(density)
    lou_expansion_values.append(expansion)
    lou_conductance_values.append(conductance)

In [35]:
print("Density: ",np.mean(lou_density_values))
print("Expansion: ",np.mean(lou_expansion_values))
print("Conductance: ",np.mean(lou_conductance_values))

Density:  0.23267881532939308
Expansion:  0.007920300676051335
Conductance:  0.00023403739983213353


In [36]:
community_df = pd.DataFrame(columns=['Customer_ID', 'Community'])

for i, community in enumerate(communities, start=1):
    for customer_id in community:
        community_df = pd.concat([community_df, pd.DataFrame({'Customer_ID': [customer_id], 'Community': [i]})], ignore_index=True)

In [37]:
community_df

,Customer_ID,Community
0,CUST72402080,1
1,CUST14587276,1
2,CUST53118934,1
3,CUST67946529,1
4,EXTERNAL909473,1
...,...,...
3588,EXTERNAL112763,126
3589,EXTERNAL303022,127
3590,EXTERNAL304081,128
3591,EXTERNAL501077,129


In [38]:
risky_individuals = pd.read_csv("cust_id_task_2.csv")
risky_individuals23 = pd.read_csv("cust_id_task_2_3.csv")
all_names = pd.read_csv("all_names.csv")
risky_individuals_3 = pd.read_csv("cust_id_3.csv")

In [39]:
names_community = pd.merge(community_df, all_names, left_on='Customer_ID', right_on='customer_id', how='left')
names_community['Risk'] =  names_community['Customer_ID'].isin(risky_individuals['Cust_id']).astype(int)
names_community.drop(['Customer_ID','customer_id'], axis=1, inplace=True)
names_community

,Community,name,Risk
0,1,CHRISTINE ESTRADA,0
1,1,CHARLES CROSS,0
2,1,SHELLEY LONG,0
3,1,AGNÈS MALTAIS,0
4,1,FREDERICK COX,0
...,...,...,...
3588,126,DR.MAXIME CHAMPAGNE,0
3589,127,DR.TODD RUIZ,0
3590,128,REN XUE MEI,0
3591,129,SCOTT ENGLISH,0


In [40]:
result = names_community.groupby('Community').agg(Average_Risk=('Risk', 'mean'),Members=('name', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result = result.sort_values(by='Average_Risk', ascending=False)
result = result[result['Average_Risk'] != 0]
result.to_csv('risk_community.csv',index=False)

# Label Propagation

In [41]:
G = nx.DiGraph()
for index, row in edges.iterrows():
    sender = row['Source']
    receiver = row['Target']
    amount = row['Weight']
    if G.has_edge(sender, receiver):
        G[sender][receiver]['weight'] += amount
    else:
        G.add_edge(sender, receiver, weight=amount)

In [42]:
G.edges(data=True)

OutEdgeDataView([('CUST76082816', 'EXTERNAL909473', {'weight': 30608.1}), ('CUST76082816', 'CUST72402080', {'weight': 27470.4}), ('CUST76082816', 'CUST18923900', {'weight': 27397.2}), ('CUST76082816', 'EXTERNAL555295', {'weight': 7441.5}), ('CUST76082816', 'CUST14587276', {'weight': 3789.0}), ('CUST76082816', 'EXTERNAL631948', {'weight': 3538.5}), ('CUST76082816', 'CUST73333517', {'weight': 2421.0}), ('CUST76082816', 'CUST67946529', {'weight': 1551.0}), ('CUST76082816', 'CUST53118934', {'weight': 177.3}), ('CUST72402080', 'CUST46447625', {'weight': 634.65}), ('CUST29999330', 'EXTERNAL176667', {'weight': 20475.0}), ('CUST29999330', 'CUST71949332', {'weight': 220.8}), ('EXTERNAL317650', 'CUST41274277', {'weight': 19731.0}), ('EXTERNAL317650', 'CUST85769551', {'weight': 27257.1}), ('EXTERNAL317650', 'CUST49987229', {'weight': 13260.0}), ('EXTERNAL272373', 'CUST33550411', {'weight': 19086.0}), ('EXTERNAL272373', 'CUST45577461', {'weight': 907.8}), ('EXTERNAL272373', 'CUST23441523', {'weigh

In [43]:
random_seed = 42
label_propagation_communities = nx.algorithms.community.asyn_lpa_communities(G,seed=random_seed)
node_community_map = {}
for idx, community in enumerate(label_propagation_communities):
    for node in community:
        node_community_map[node] = idx

community_df = pd.DataFrame(list(node_community_map.items()), columns=['Node', 'Community'])

In [44]:
community_df
print("No. of Communities: ",len(community_df['Community'].unique()))

No. of Communities:  943


In [45]:
random_seed = 42
label_propagation_communities = nx.algorithms.community.asyn_lpa_communities(G,seed=random_seed)

lab_density_values = []
lab_expansion_values = []
lab_conductance_values = []

for community in label_propagation_communities:
    density = compute_density(edgelist_plot, community)
    expansion = compute_expansion(edgelist_plot, community)
    conductance = compute_conductance(edgelist_plot, community)
    lab_density_values.append(density)
    lab_expansion_values.append(expansion)
    lab_conductance_values.append(conductance)

print("Density: ",np.mean(lab_density_values))
print("Expansion: ",np.mean(lab_expansion_values))
print("Conductance: ",np.mean(lab_conductance_values))

Density:  0.35556862200204253
Expansion:  0.001063809113706764
Conductance:  0.00015420720422460883


In [46]:
names_community = pd.merge(community_df, all_names, left_on='Node', right_on='customer_id', how='left')
names_community['Risk'] =  names_community['Node'].isin(risky_individuals['Cust_id']).astype(int)
names_community.drop(['Node'], axis=1, inplace=True)
names_community

,Community,customer_id,name,Risk
0,0,CUST73333517,DR.VALERIE CARPENTER,0
1,0,CUST76082816,STEVEN YOUNG,0
2,1,EXTERNAL909473,FREDERICK COX,0
3,2,CUST72402080,CHRISTINE ESTRADA,0
4,2,EXTERNAL147495,DIANA HUGHES,0
...,...,...,...,...
3588,940,EXTERNAL323134,SUN GUI ZHI,0
3589,941,EXTERNAL413952,DR.CAROLINE RICHARDSON,0
3590,942,CUST22403161,CRAIG GREGORY DDS,0
3591,942,CUST72752478,SÉBASTIEN LAMONTAGNE-BRUNET,0


In [62]:
import requests
from bs4 import BeautifulSoup
def os_query_check(names):
    results = []
    for name in names:
        name_reformed = name.replace(" ", "+")
        api_url = "https://www.opensanctions.org/search/?q="+name_reformed
        response = requests.get(api_url)
    
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        
            alert_div = soup.find("div", class_="alert-heading")
        
            if alert_div and "No matching entities were found." in alert_div.text:
                results.append(0)
            else:
                results.append(1)
    return results

In [63]:
result = names_community.groupby('Community').agg(Average_Risk=('Risk', 'mean'),Members=('name', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result = result.sort_values(by='Average_Risk', ascending=False)
result = result[result['Average_Risk'] != 0]
result.to_csv('risk_community_directed.csv',index=False)

In [64]:
len(result)

41

In [65]:
result_df_filtered = names_community[names_community["Community"].isin(result["Community"])]

In [66]:
result_df_filtered

,Community,customer_id,name,Risk
38,8,EXTERNAL316517,JONATHAN SANTANA,0
39,8,EXTERNAL273233,KIM GREGORY,0
40,8,CUST54692597,ANA LEBRÓN TRUJILLO,0
41,8,EXTERNAL925297,GENEVIÈVE JACQUES-BERNIER,0
42,8,CUST83823297,BENJAMIN NELSON,0
...,...,...,...,...
2571,533,CUST47035199,REBECCA HANEY,1
2708,575,CUST46864961,JASON MCINTOSH,1
2709,575,EXTERNAL200776,JONATHAN TAYLOR,0
2710,575,CUST44498716,SARA DAVE,0


In [71]:
result_df_filtered['OS_Results'] = os_query_check(result_df_filtered['name'])

In [69]:
result = result_df_filtered.groupby('Community').agg(Average_Risk=('Combined_Result', 'mean'),Members=('name', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result = result.sort_values(by='Average_Risk', ascending=False)
result = result[result['Average_Risk'] != 0]
result.to_csv('risk_community_directed.csv',index=False)

In [70]:
result

,Community,Average_Risk,Members,Num_Members
25,309,1.000000,SAMANTHA OWEN,1
3,24,1.000000,ERMIL VILENOVICH VLASOV,1
5,26,1.000000,MICHAEL PAGE,1
6,35,1.000000,"CASSIE WASHINGTON, ANNE DANIEL",2
19,204,1.000000,LANGALIBALELE SHOZI,1
16,181,0.750000,"DR.JOHN WADE, ZACHARIE GRENIER, MARTHA KING, B...",12
27,351,0.666667,"ELAINE GRANT, DR.EDWARD FUENTES, COREY JACKSON",3
2,23,0.666667,"TODD DOYLE, GUO XIAO HONG, STEVEN CRUZ",3
29,383,0.666667,"WHITNEY WALTON, WANG LONG, KIMBERLY BROCK",3
9,59,0.666667,"WANG XIU MEI, JOANN ASHLEY, JIMMY DOMINGUEZ, S...",15


In [55]:
filtered_df_new1 = result_df_filtered[(result_df_filtered["Risk"] == 1) | (result_df_filtered["OS_Results"] == 1)]

filtered_df_new1.reset_index(drop=True, inplace=True)
filtered_df_new1

,Community,customer_id,name,Risk,OS_Results,Combined_Result
0,8,EXTERNAL316517,JONATHAN SANTANA,0,1,1
1,8,CUST83823297,BENJAMIN NELSON,0,1,1
2,8,EXTERNAL493741,SUSAN RODRIGUEZ,0,1,1
3,8,CUST24401026,PATRICIA RUSSELL,0,1,1
4,8,EXTERNAL439731,DAKOTA MILLER,1,1,1
...,...,...,...,...,...,...
156,510,CUST75429953,VINCENT PEREZ,1,1,1
157,533,EXTERNAL622718,LIU HUI,0,1,1
158,533,CUST47035199,REBECCA HANEY,1,0,1
159,575,CUST46864961,JASON MCINTOSH,1,0,1


In [57]:
final_list = pd.merge(filtered_df_new1, result, on='Community', how='left')

In [58]:
final_list

,Community,customer_id,name,Risk,OS_Results,Combined_Result,Average_Risk,Members,Num_Members
0,8,EXTERNAL316517,JONATHAN SANTANA,0,1,1,0.384615,"JONATHAN SANTANA, KIM GREGORY, ANA LEBRÓN TRUJ...",26
1,8,CUST83823297,BENJAMIN NELSON,0,1,1,0.384615,"JONATHAN SANTANA, KIM GREGORY, ANA LEBRÓN TRUJ...",26
2,8,EXTERNAL493741,SUSAN RODRIGUEZ,0,1,1,0.384615,"JONATHAN SANTANA, KIM GREGORY, ANA LEBRÓN TRUJ...",26
3,8,CUST24401026,PATRICIA RUSSELL,0,1,1,0.384615,"JONATHAN SANTANA, KIM GREGORY, ANA LEBRÓN TRUJ...",26
4,8,EXTERNAL439731,DAKOTA MILLER,1,1,1,0.384615,"JONATHAN SANTANA, KIM GREGORY, ANA LEBRÓN TRUJ...",26
...,...,...,...,...,...,...,...,...,...
156,510,CUST75429953,VINCENT PEREZ,1,1,1,0.166667,"LUC LAPIERRE, RAPHAËL LAPOINTE, LUKE SCHNEIDER...",6
157,533,EXTERNAL622718,LIU HUI,0,1,1,0.285714,"TIFFANY BOWEN, LIU HUI, GEORGES LANDRY, DR.CRY...",7
158,533,CUST47035199,REBECCA HANEY,1,0,1,0.285714,"TIFFANY BOWEN, LIU HUI, GEORGES LANDRY, DR.CRY...",7
159,575,CUST46864961,JASON MCINTOSH,1,0,1,0.500000,"JASON MCINTOSH, JONATHAN TAYLOR, SARA DAVE, ÉL...",4


In [61]:
final_list[['customer_id','name','Community','Average_Risk']].to_csv("community_risk_task2.csv.")

In [26]:
# Task 2 + Task 3
edges = pd.read_csv('group_23.csv')


In [27]:
G2 = nx.DiGraph()
for index, row in edges.iterrows():
    sender = row['Source']
    receiver = row['Target']
    amount = row['Weight']
    if G2.has_edge(sender, receiver):
        G2[sender][receiver]['weight'] += amount
    else:
        G2.add_edge(sender, receiver, weight=amount)

In [28]:
random_seed = 42
label_propagation_communities2 = nx.algorithms.community.asyn_lpa_communities(G2,seed=random_seed)
node_community_map2 = {}
for idx, community in enumerate(label_propagation_communities2):
    for node in community:
        node_community_map2[node] = idx

community_df2 = pd.DataFrame(list(node_community_map2.items()), columns=['Node', 'Community'])

In [29]:
community_df2
print("No. of Communities: ",len(community_df2['Community'].unique()))

No. of Communities:  1306


In [ ]:
names_community2 = pd.merge(community_df2, all_names, left_on='Node', right_on='customer_id', how='left')
names_community2['Risk'] =  names_community2['Node'].isin(risky_individuals23['Cust_id']).astype(int)
names_community2.drop(['Node'], axis=1, inplace=True)
names_community2

In [ ]:
result2 = names_community2.groupby('Community').agg(Average_Risk=('Risk', 'mean'),Members=('name', lambda x: ', '.join(x)),IDs=('customer_id', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result2 = result2.sort_values(by='Average_Risk', ascending=False)
result2 = result2[result2['Average_Risk'] != 0]
result2.to_csv('risk_community_directed_task3.csv',index=False)

In [ ]:
len(result2)

In [ ]:
result2['Community']

In [ ]:
result2_df_filtered = names_community2[names_community2["Community"].isin(result2["Community"])]

In [ ]:
result2_df_filtered

In [ ]:
result2_df_filtered['OS_Results'] = os_query_check(result2_df_filtered['name'])

In [ ]:
result2_df_filtered['Combined_Result'] = result2_df_filtered[['Risk', 'OS_Results']].max(axis=1)

In [ ]:
result2_df_filtered

In [ ]:
result2_3 = result2_df_filtered.groupby('Community').agg(Average_Risk=('Combined_Result', 'mean'),Members=('name', lambda x: ', '.join(x)),IDs=('customer_id', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result2_3 = result2_3.sort_values(by='Average_Risk', ascending=False)
result2_3 = result2_3[result2_3['Average_Risk'] != 0]
result2_3.to_csv('risk_community_directed_task3.csv',index=False)

In [ ]:
filtered_df_new = result2_df_filtered[(result2_df_filtered["Risk"] == 1) | (result2_df_filtered["OS_Results"] == 1)]

filtered_df_new.reset_index(drop=True, inplace=True)
filtered_df_new

# Community Analysis

In [11]:
com_data = names_community[names_community['Community']==59]
com_data

,Community,name,Risk
1656,59,CYNTHIA WHITE,0
1657,59,KELLY SIMMONS,0
1658,59,KEVIN PAUL,0
1659,59,JANICE WOODS,0
1660,59,ANGELA MARTINEZ,0
1661,59,COURTNEY HILL,0
1662,59,JENNIFER HANSEN,0
1663,59,KEVIN DAVIS,0
1664,59,CHARLOTTE BRUNET-GUAY,0
1665,59,MICHAEL CLINE,0


In [12]:
kyc = pd.read_csv("kyc_full_102_columns.csv")

In [ ]:
kyc

In [13]:
kyc2 = pd.read_csv("kyc_with_open_sanctions.csv")

In [ ]:
kyc2

In [14]:
com_data = pd.merge(com_data, kyc2, left_on='customer_id', right_on='cust_id', how='left')

KeyError: 'customer_id'

In [ ]:
com_data = pd.merge(com_data, kyc, left_on='customer_id', right_on='cust_id', how='left')

In [ ]:
com_data.columns

In [ ]:
com_data = com_data.drop(columns=['Community','name','cust_id_x','Gender_y', 'Age_y', 'Tenure_y', 'cust_id_y',
       'OpenSanc_query_y',
       'label_y',
       'wildlife_occupation', 'trafficking_occupation',
       'transportation_occupation', 'media_occupation', 'law_occupation','wire_trxn_sent_to', 'wire_trxn_received_from', 'average_wire_sent',
       'average_wire_received', 'no_external_wire_trxn_received',
       'no_external_wire_trxn_sent', 'no_wire_wildlife_occupation_sent',
       'no_wire_wildlife_occupation_received',
       'no_wire_trafficking_occupation_sent',
       'no_wire_trafficking_occupation_received',
       'no_wire_transportation_occupation_sent',
       'no_wire_transportation_occupation_received',
       'no_wire_media_occupation_sent', 'no_wire_media_occupation_received',
       'no_wire_law_occupation_sent', 'no_wire_law_occupation_received','emt_trxn_sent_to_msg',
       'emt_trxn_received_from_msg', 'emt_trxn_sent_to',
       'emt_trxn_received_from', 'average_emt_sent', 'average_emt_received',
       'no_external_emt_trxn_received', 'no_external_emt_trxn_sent',
       'no_emt_wildlife_occupation_sent',
       'no_emt_wildlife_occupation_received',
       'no_emt_trafficking_occupation_sent',
       'no_emt_trafficking_occupation_received',
       'no_emt_transportation_occupation_sent',
       'no_emt_transportation_occupation_received',
       'no_emt_media_occupation_sent', 'no_emt_media_occupation_received',
       'no_emt_law_occupation_sent', 'no_emt_law_occupation_received','no_emt_transportation_msg_sent', 'no_emt_transportation_msg_received',
       'no_emt_law_msg_sent', 'no_emt_law_msg_received',
       'no_emt_trafficking_msg_sent', 'no_emt_trafficking_msg_received',
       'no_emt_accomodation_msg_sent', 'no_emt_accomodation_msg_received'])

In [ ]:
com_data

In [ ]:
com_data.to_csv('community_data.csv',index=False)

# Plotting

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
from IPython.core.display import display, HTML

# Assuming you have already created your graph edgelist_plot and computed the communities
# Let's assume communities is a list of sets where each set represents a community

# Convert the list of communities into a dictionary mapping each node to its community

G = nx.DiGraph()
for index, row in edges.iterrows():
    sender = row['Source']
    receiver = row['Target']
    amount = row['Weight']
    if G.has_edge(sender, receiver):
        G[sender][receiver]['weight'] += amount
    else:
        G.add_edge(sender, receiver, weight=amount)
        
random_seed = 42
label_propagation_communities = nx.algorithms.community.asyn_lpa_communities(G,seed=random_seed)
node_community_map = {}
for idx, community in enumerate(label_propagation_communities):
    for node in community:
        node_community_map[node] = idx + 1  # Adjust index to start from 1

# Find nodes belonging to community 59
community_59_nodes = [node for node, community in node_community_map.items() if community == 60]

edgelist_plot = nx.from_pandas_edgelist(edges, source='Source', target='Target', edge_attr=["Weight"], create_using=nx.DiGraph())

# Extract edges associated with these nodes
community_59_edges = [(source, target, weight) for source, target, weight in edgelist_plot.edges(data=True) if source in community_59_nodes or target in community_59_nodes]


# Create a new graph for community 59
community_59_graph = nx.DiGraph()
community_59_graph.add_edges_from(community_59_edges)

id_to_name = dict(zip(all_names['customer_id'], all_names['name']))
nodes_with_names = [id_to_name[node] if node in id_to_name else node for node in community_59_nodes]
nx.set_node_attributes(community_59_graph, id_to_name, 'label')

nt = Network(notebook=True, directed=True,cdn_resources='remote', bgcolor="#ffffff", font_color="black")

nt.from_nx(community_59_graph)

for node in nt.nodes:
        node['color'] = 'red'

nt.show('community_59.html')

In [ ]:
community_59_edges

In [ ]:
nodes_with_names

# Suspicious Customers Related

In [31]:
names_community3 = pd.merge(community_df2, all_names, left_on='Node', right_on='customer_id', how='left')
names_community3['Risk'] =  names_community3['Node'].isin(risky_individuals_3['Cust_id']).astype(int)
names_community3.drop(['Node'], axis=1, inplace=True)
names_community3

,Community,customer_id,name,Risk
0,0,CUST68585930,BARBARA JACKSON,0
1,0,EXTERNAL939562,HERBERT DANIELS,0
2,0,EXTERNAL594322,AURORE LESSARD,0
3,0,EXTERNAL802264,KIM BAUTISTA,0
4,0,EXTERNAL144308,DR.RUBEN WHITE,0
...,...,...,...,...
4925,1302,CUST98072377,MISS SAMANTHA WILLIAMSON,0
4926,1303,EXTERNAL995220,KAREN ADAMS,0
4927,1303,CUST74270470,STEPHANIE LARSEN,0
4928,1304,CUST50579675,COURTNEY LEE,0


In [32]:
result3 = names_community3.groupby('Community').agg(Average_Risk=('Risk', 'mean'),Members=('name', lambda x: ', '.join(x)),IDs=('customer_id', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result3 = result3.sort_values(by='Average_Risk', ascending=False)
result3 = result3[result3['Average_Risk'] != 0]


In [33]:
result3_df_filtered = names_community3[names_community3["Community"].isin(result3["Community"])]

In [34]:
result3_df_filtered

,Community,customer_id,name,Risk
131,10,CUST95429262,DR.MARIO VANCE,0
132,10,CUST10465526,KAYLA MARTINEZ,0
133,10,CUST92468556,DR.SHARON MASON,0
134,10,CUST19678855,AMBER LUCAS,0
135,10,EXTERNAL931239,THOMAS DESBIENS,0
...,...,...,...,...
4714,1149,CUST77329011,MARK HARRISON,1
4772,1189,CUST38730398,MICHEL LAROSE,1
4773,1189,EXTERNAL573040,MARGARET CARTER,0
4833,1234,CUST56065075,WANG QIANG,1


In [35]:
result3_df_filtered['OS_Results'] = os_query_check(result3_df_filtered['name'])

In [36]:
result3_df_filtered['Combined_Result'] = result3_df_filtered[['Risk', 'OS_Results']].max(axis=1)

In [37]:
result3 = result3_df_filtered.groupby('Community').agg(Average_Risk=('Combined_Result', 'mean'),Members=('name', lambda x: ', '.join(x)),IDs=('customer_id', lambda x: ', '.join(x)),Num_Members=('name', 'count')).reset_index()
result3 = result3.sort_values(by='Average_Risk', ascending=False)
result3 = result3[result3['Average_Risk'] != 0]
result3.to_csv('risk_community_directed_task3_alone.csv',index=False)

In [38]:
filtered_df_new3 = result3_df_filtered[(result3_df_filtered["Risk"] == 1) | (result3_df_filtered["OS_Results"] == 1)]

filtered_df_new3.reset_index(drop=True, inplace=True)
filtered_df_new3

,Community,customer_id,name,Risk,OS_Results,Combined_Result
0,10,CUST66939693,XIANG JUN,0,1,1
1,10,CUST92670137,TIMOTHY JOHNSON,0,1,1
2,10,CUST36687448,MA JIAN,0,1,1
3,10,CUST67032027,PAMELA THOMAS,0,1,1
4,10,EXTERNAL531933,RYAN VAUGHN,1,0,1
...,...,...,...,...,...,...
105,922,EXTERNAL399093,PATRICIA MARTIN,0,1,1
106,1149,EXTERNAL401777,MELISSA PRICE,0,1,1
107,1149,CUST77329011,MARK HARRISON,1,0,1
108,1189,CUST38730398,MICHEL LAROSE,1,0,1


In [39]:
filtered_df_new3['customer_id'].to_csv("Team#37_task3_2_IDs.csv.")